In [2]:
import pandas as pd
import spacy
import re
import tldextract
import tensorflow as tf
import numpy as np

nlp = spacy.load('en_core_web_sm')

STOP_WORDS = nlp.Defaults.stop_words

nRowsRead = None  # specify 'None' if want to read whole file
postStrings = []
typeStrings = []


2022-09-04 23:19:31.633409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-04 23:19:31.633435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-04 23:19:33.339664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-04 23:19:33.339687: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-04 23:19:33.339715: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marko-pc): /proc/driver/nvidia/version does not exist


In [18]:
df = pd.read_csv('./input/mbti_1.csv', delimiter=',', nrows=nRowsRead)
df.dataframeName = 'mbti_1.csv'

In [19]:
# making array of array of posts
for person,type in zip(df['posts'],df['type']):
    postStrings.append([type,person.split('|||')])

# removing excess single parentheses

for i in range(0,len(postStrings)):
    postStrings[i][1][0] = postStrings[i][1][0][1:]
    postStrings[i][1][-1] = postStrings[i][1][-1][:-1]

In [20]:
#splitting posts into list of posts
df['posts'] = df['posts'].apply(lambda x: x.split('|||'))

# removing excess single parentheses
def removeParentheses(posts):
    posts[0] = posts[0][1:]
    posts[-1] = posts[-1][:-1]
    return posts
df['posts'] = df['posts'].apply(lambda x: removeParentheses(x))

df.head()

,type,posts
0,INFJ,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,ENTP,[I'm finding the lack of me in these posts ver...
2,INTP,[Good one _____ https://www.youtube.com/wat...
3,INTJ,"[Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,"[You're fired., That's another silly misconcep..."


In [21]:
#label encoding type

def labelEncodeType(type):
  if type == 'ISTJ' or type == 'ISFJ' or type == 'ESTJ' or type == 'ESFJ':
    type = 0 #GUARDIAN
  elif type == 'ISTP' or type == 'ISFP' or type == 'ESTP' or type == 'ESFP':
    type = 1 #ARTISAN
  elif type == 'INFJ' or type == 'INFP' or type == 'ENFP' or type == 'ENFJ':
    type = 2  # IDEALIST
  elif type == 'INTJ' or type == 'INTP' or type == 'ENTP' or type == 'ENTJ':
    type = 3  # RATIONALIST
  return type

df['type'] = df['type'].apply(lambda x: labelEncodeType(x))
df.head()

,type,posts
0,2,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,3,[I'm finding the lack of me in these posts ver...
2,3,[Good one _____ https://www.youtube.com/wat...
3,3,"[Dear INTP, I enjoyed our conversation the o..."
4,3,"[You're fired., That's another silly misconcep..."


In [22]:
df['type'].value_counts()

2    4167
3    3311
1     745
0     452
Name: type, dtype: int64

In [23]:
URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
DOMAIN_REGEX = "^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/?\n]+)"
COMMENT_REGEX = r"\[.*?\]"
ELLIPSIS_REGEX = r"^(\ *\.{3}\ *)|(\ *\.{3}\ *)$"

def urlReplace(stringToReplace):
	domain = tldextract.extract(stringToReplace.group())
	return domain.domain + '.' + domain.suffix


**CONSIDER TRYING WITHOUT STOPWORD REMOVAL**

In [24]:
def containsOnlyUrlsOrNumbers(post):
	splits = post.split()
	for str in splits:
		if(not re.search(URL_REGEX,str) and not re.search('^\d+$',str) and not re.search('^\.\.\.+$',str)):
			return 1
	return 0

def containsComments(post):
	splits = post.split('] ')
	for str in splits:
		if(not re.search(COMMENT_REGEX,str)):
			return 1
		return 0

def removeEllipsis(post):
	return re.sub(ELLIPSIS_REGEX,'',post)

def removeWhitespace(post):
	post = " ".join(post.split())
	return post

def removeStopWords(post):
	post = " ".join([t for t in post.split() if t not in STOP_WORDS])
	return post

def formatPerson(person):
	person = list(map(lambda x: removeWhitespace(x), person))
	person = [post for post in person if (
		containsOnlyUrlsOrNumbers(post) and containsComments(post))]
	person = [removeEllipsis(post) for post in person]
	person = list(map(lambda x: re.sub(URL_REGEX, urlReplace, x), person))
	person = list(map(lambda x: removeStopWords(x),person))
	return person

df['posts']= df['posts'].apply(lambda x:formatPerson(x))

df.head()

,type,posts
0,2,[enfp intj moments youtube.com sportscenter pl...
1,3,"[I'm finding lack posts alarming., Sex boring ..."
2,3,"[Good _____ youtube.com, Of course, I I know; ..."
3,3,"[Dear INTP, I enjoyed conversation day. Esoter..."
4,3,"[You're fired., That's silly misconception. Th..."


In [25]:
#do the nlp tokenization with spaCy

def convertToLemmas(posts):
  docs=nlp.pipe(posts,n_process=-1)
  lemmaList = []
  postList = []
  for doc in docs:
    for token in doc:
      lemma = str(token.lemma_)
      if lemma == '-PRON-' or lemma == 'be':
        lemma = token.text
      lemmaList.append(lemma)
    postList.append(" ".join(lemmaList))
    lemmaList= []
  return postList

#for the dataframe:
df['posts']= df['posts'].apply(lambda x:convertToLemmas(x))
df.head()

,type,posts
0,2,[enfp intj moments youtube.com sportscenter pl...
1,3,"[I 'm find lack post alarm ., sex boring it 's..."
2,3,"[good _ _ _ _ _ youtube.com, of course , I I k..."
3,3,"[Dear INTP , I enjoy conversation day . Esoter..."
4,3,"[you 're fire ., that 's silly misconception ...."


In [26]:
#SPLITTING POSTS INTO SEPERATE ROWS
df = df.explode('posts')
df

,type,posts
0,2,enfp intj moments youtube.com sportscenter pla...
0,2,what life - change experience life ?
0,2,youtube.com youtube.com on repeat today .
0,2,May PerC experience immerse you .
0,2,the thing infj friend post facebook commit sui...
...,...,...
8674,2,"I go close facebook month back , want able mes..."
8674,2,30 Seconds Mars - all collection . it fitting ...
8674,2,"I see it , agree . I actually think time I wat..."
8674,2,ok watch Underworld 4 ( Awakening ) good film ...


In [27]:
df['type'].value_counts()

2    197249
3    155773
1     34619
0     21308
Name: type, dtype: int64

In [28]:
#common words removal
freq_comm = pd.Series(' '.join(df['posts'].tolist()).split()).value_counts()
f20= freq_comm[:20]

In [29]:
df['posts'] = df['posts'].apply(lambda x: ' '.join([t for t in x.split() if t not in f20]))

In [30]:
#rare words removal
rare= freq_comm[freq_comm.values==1]

In [31]:
df['posts'] = df['posts'].apply(lambda x: ' '.join([t for t in x.split() if t not in rare]))

In [4]:
df0 = df[df['type'] == 0].sample(10000)
df1 = df[df['type'] == 1].sample(10000)
df2 = df[df['type'] == 2].sample(10000)
df3 = df[df['type'] == 3].sample(10000)

dfr = pd.concat([df0,df1,df2,df3],)
dfr

,type,posts
4196,0,sometimes actually ISFP
3603,0,outdoors usually prefer hike long walk else
2234,0,intp understand pretty too guess hard person
87,0,wow finally peopl can stand Jacob all read boo...
3701,0,hm ISFP youtube.com
...,...,...
3469,3,see sort help bit perception function opposite...
2545,3,sometimes usually argument debate excited fact...
7710,3,bit pickle my dad drop college cost money GPA ...
2585,3,hi everyone ask now ENTP Are emotional moron l...


In [35]:
# df.to_csv(path_or_buf="./csv_files/df.csv")
# dfr.to_csv(path_or_buf="./csv_files/dfr.csv")

In [5]:
y = dfr['type']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv = CountVectorizer(dtype='b')
text_counts = cv.fit_transform(dfr['posts'])

In [15]:
text_counts.toarray().shape

(40000, 22250)

In [16]:
dfr_bow = pd.DataFrame(text_counts.toarray(),columns=cv.get_feature_names_out())

In [31]:
dfr_bow.head(2)

,00,000,000th,006,00am,01,02,03,0493,06,...,zzzz,zzzzz,âx80x93,âx80x9cthe,är,ça,éowyn,ête,über,사랑
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# dfr_bow.to_csv(path_or_buf="./csv_files/dfr_bow.csv")

## ML ALGORITHMS

In [32]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [33]:
rfc = RandomForestClassifier(random_state=42,n_estimators=500,n_jobs=-1,max_depth=8)

In [34]:
%%time

def classify(X,y):
  scaler = MinMaxScaler(feature_range=(0,1))
  X=scaler.fit_transform(X)

  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.2,random_state=42,stratify=y)

  rfc.fit(X_train,y_train)
  y_pred = rfc.predict(X_test)
  ac = accuracy_score(y_test,y_pred)
  print("accuracy: ",ac)


classify(dfr_bow,y)

accuracy:  0.35475
CPU times: user 7min 54s, sys: 54 s, total: 8min 48s
Wall time: 1min 57s
